## Fine-Tuning Neural Network Hyperparameters
 Using Keras_Tuner library, which is a hyperparameter tuning library for Keras models.Here are general steps:
 1. import keras_tuner as kt. ( or first install the library: pip install keras_tuner)
 2. write a function that builds, compiles and returns Keras model. (The function must take a kt.Hyperparameters object as an argument, which it can use ot define hyperparameters(inters,floats,string,etc.)along with their range of possible values. 
 3. using kt.RandomSearch() or kt.Hyperband()tuner, passing build_model function to the constructor.
 4. Call the tuner's search() method
 5. tuner.get_best_models(num_models=n)(get the best model),get_best_hyperparameters(num_trials=d)( to get the best parameters)
 
 
 

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras_tuner import RandomSearch
import keras_tuner as kt

# Sample data
input1 = np.random.rand(1000, 8)
input2 = np.random.rand(1000, 8)
output1 = np.random.rand(1000, 1)
output2 = np.random.rand(1000, 1)

# Model definition with Normalization layers
def build_model(hp):
    input_a = keras.layers.Input(shape=(8,), name='input_a')
    normalized_a = keras.layers.Normalization()(input_a)

    input_b = keras.layers.Input(shape=(8,), name='input_b')
    normalized_b = keras.layers.Normalization()(input_b)

    x1 = keras.layers.Dense(units=hp.Int('dense_1_units', min_value=32, max_value=128, step=32), activation='relu')(normalized_a)
    x2 = keras.layers.Dense(units=hp.Int('dense_2_units', min_value=32, max_value=128, step=32), activation='relu')(normalized_b)
    
    merged = keras.layers.concatenate([x1, x2])
    out_a = keras.layers.Dense(1, name='out_a')(merged)
    out_b = keras.layers.Dense(1, name='out_b')(merged)

    model = keras.models.Model(inputs=[input_a, input_b], outputs=[out_a, out_b])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss={'out_a': 'mse', 'out_b': 'mse'},
                  metrics={'out_a': 'mae', 'out_b': 'mae'})
    return model

# Keras Tuner setup
tuner = RandomSearch(
    build_model,
    objective=kt.Objective("val_out_a_mae", direction="min"),
    overwrite=True,
    max_trials=5,
    executions_per_trial=3,
    directory='output_dir',
    project_name='keras_tuner_example'
)

# Search for the best hyperparameters
tuner.search([input1, input2], [output1, output2], epochs=5, validation_split=0.2)

# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Adapt the normalization layers in the best model
normalization_layer_a = best_model.get_layer(name='normalization')  # Note: Keras automatically appends an index to the layer name
normalization_layer_a.adapt(input1)

normalization_layer_b = best_model.get_layer(name='normalization_1')
normalization_layer_b.adapt(input2)

# Fit the best model to the data
history = best_model.fit([input1, input2], [output1, output2], epochs=10, validation_split=0.2)


Trial 5 Complete [00h 00m 08s]
val_out_a_mae: 0.2617969314257304

Best val_out_a_mae So Far: 0.25833870967229206
Total elapsed time: 00h 00m 40s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
25/25 [==============================] - 2s 16ms/step - loss: 0.2681 - out_a_loss: 0.1530 - out_b_loss: 0.1151 - out_a_mae: 0.3176 - out_b_mae: 0.2777 - val_loss: 0.2143 - val_out_a_loss: 0.1031 - val_out_b_loss: 0.1112 - val_out_a_mae: 0.2675 - val_out_b_mae: 0.2813
Epoch 2/10
25/25 [==============================] - 0s 5ms/step - loss: 0.1843 - out_a_loss: 0.0893 - out_b_loss: 0.0950 - out_a_mae: 0.2476 - out_b_mae: 0.2557 - val_loss: 0.1912 - val_out_a_loss: 0.0958 - val_out_b_loss: 0.0955 - val_out_a_mae: 0.2582 - val_out_b_mae: 0.2606
Epoch 3/10
25/25 [==============================] - 0s 4ms/step - loss: 0.1715 - out_a_loss: 0.0825 - out_b_loss: 0.0890 - out_a_mae: 0.2418 - out_b_mae: 0.2505 - val_loss: 0.1922 - val_out_a_loss: 0.0928 - val_out_b_loss: 0.0994 - val_out_a_mae: 0.2606 - val_

In [2]:
from keras.datasets import fashion_mnist

In [3]:
(x_train_full,y_train_full),(x_test,y_test) = fashion_mnist.load_data()

In [4]:
x_train,x_val = x_train_full[5000:]/255,x_train_full[:5000]/255

In [5]:
y_train,y_val = y_train_full[5000:],y_train_full[:5000]

In [6]:
def build_mod(hp):
    input_ = keras.layers.Input(shape=[28,28])
    flatten= keras.layers.Flatten()(input_)
    hidden1=keras.layers.Dense(units=hp.Int('dense_1',min_value=30,max_value=300,step=30),activation='relu')(flatten)
    hidden2=keras.layers.Dense(units=hp.Int('dense_2',min_value=30,max_value=300,step=30),activation='relu')(hidden1)
    output = keras.layers.Dense(10,activation='softmax')(hidden2)
    optimizer = hp.Choice('optimizer',values=['sgd','adam'])
  
    if optimizer =='sgd':
        optimizer = keras.optimizers.SGD(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    else:
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    model = keras.Model(inputs=[input_],outputs=[output])
    model.compile(loss='sparse_categorical_crossentropy',
                 optimizer = optimizer,
                 metrics=['accuracy'])
    return model
    
    
    

In [7]:
tuner = kt.RandomSearch(build_mod,
                     objective='val_accuracy',
                     overwrite=True,
                     max_trials=5,
                     executions_per_trial=3,
                     directory='output_diry',
                     project_name='keras_tuner_example_1',
                    )

In [8]:
tuner.search(x_train,y_train,epochs=10,validation_data=(x_val,y_val))

Trial 5 Complete [00h 02m 15s]
val_accuracy: 0.8733999927838644

Best val_accuracy So Far: 0.8891333142916361
Total elapsed time: 00h 13m 45s
INFO:tensorflow:Oracle triggered exit


In [9]:
top3_params = tuner.get_best_hyperparameters(num_trials=3)


In [10]:
top3_params[0].values

{'dense_1': 300, 'dense_2': 90, 'optimizer': 'adam', 'learning_rate': 0.0001}

In [11]:
top3_model = tuner.get_best_models(num_models=3)
best_model = top3_model[0]


In [12]:
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 3 summary
Hyperparameters:
dense_1: 300
dense_2: 90
optimizer: adam
learning_rate: 0.0001
Score: 0.8891333142916361


In [13]:
h = best_model.fit(x_train,y_train,epochs=10,validation_data=(x_val,y_val))

Epoch 1/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.2642 - accuracy: 0.9039 - val_loss: 0.3057 - val_accuracy: 0.8926
Epoch 2/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.2539 - accuracy: 0.9084 - val_loss: 0.3085 - val_accuracy: 0.8866
Epoch 3/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2477 - accuracy: 0.9109 - val_loss: 0.2979 - val_accuracy: 0.8964
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2393 - accuracy: 0.9138 - val_loss: 0.3149 - val_accuracy: 0.8896
Epoch 5/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2330 - accuracy: 0.9153 - val_loss: 0.2881 - val_accuracy: 0.8982
Epoch 6/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2249 - accuracy: 0.9189 - val_loss: 0.2905 - val_accuracy: 0.8928
Epoch 7/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.2199 - accuracy: 0.9198 - val_loss: 0.2946 - val_accuracy:

In [14]:
import pandas as pd
dd = pd.DataFrame(h.history)

In [15]:
dd


,loss,accuracy,val_loss,val_accuracy
0,0.264156,0.903927,0.305677,0.8926
1,0.253927,0.908364,0.308514,0.8866
2,0.247709,0.910873,0.297911,0.8964
3,0.239277,0.913836,0.314887,0.8896
4,0.233002,0.915273,0.288073,0.8982
5,0.224908,0.918873,0.290469,0.8928
6,0.219940,0.919818,0.294636,0.8902
7,0.214086,0.923018,0.286353,0.8990
8,0.207976,0.925164,0.281780,0.8996
9,0.203862,0.925709,0.280945,0.9002


In [16]:
best_model.evaluate(x_test,y_test)

313/313 [==============================] - 1s 2ms/step - loss: 54.8486 - accuracy: 0.8753


[54.848575592041016, 0.8752999901771545]